In [1]:
from collections import defaultdict
from tqdm.notebook import tqdm

In [2]:
data = open("input/10").read().splitlines()

In [3]:
pipes = {
    "-": [[0, 1], [0, -1]],
    "|": [[1, 0], [-1, 0]],
    "F": [[0, 1], [1, 0]],
    "7": [[1, 0], [0, -1]],
    "L": [[-1, 0], [0, 1]],
    "J": [[-1, 0], [0, -1]],
    "S": [[0, 1], [0, -1], [1, 0], [-1, 0]],
}

## Map up the pipe

In [4]:
pipe_map = defaultdict(list)
all_locations = set()
S_LOC = None
for r, line in enumerate(data):
    for c, char in enumerate(line):
        all_locations.add((r, c))
        if char == ".":
            continue
        if char == "S":
            S_LOC = (r, c)
        
        for rr, cc in pipes[char]:
            pipe_map[(r, c)].append((r + rr, c + cc))    

## Remove S's broken neighbours

In [5]:
pipe_map[S_LOC] = [node for node in pipe_map[S_LOC] if S_LOC in pipe_map[node]]

## Walk in pipe

In [6]:
loop = [S_LOC, pipe_map[S_LOC][0]]
cur = loop[-1]
while cur != S_LOC:    
    n1, n2 = pipe_map[cur]
    cur = n2 if n1 == loop[-2] else n1
    loop.append(cur)

In [7]:
print("Answer #1:", len(loop) // 2)

Answer #1: 6931


# Part 2

## Idea is to generate a finer grid with positions in between all others
* Then from all nodes to DFS until I get outside of the grid
* Need to first extend my original loop with the in between steps
* Then iterate over all nodes that are not in the loop to see if I can get out

## Generate the more detail loop with positions in between

In [8]:
loop_fine = set()
for idx, elem in enumerate(loop[:-1]):
    loop_fine.add(elem)
        
    x1, y1 = elem
    x2, y2 = loop[idx + 1]
    newx, newy = x1, y1
    
    if x1 == x2:
        newy = y1 - 0.5 if y1 > y2 else y1 + 0.5
    else:
        newx = x1 - 0.5 if x1 > x2 else x1 + 0.5

    loop_fine.add((newx, newy))

## Find all neighbours that are not in the loop

In [9]:
def find_neighbours(node):
    neighbours = []
    for row, col in [[0, 0.5], [0, -0.5], [0.5, 0], [-0.5, 0]]:
        pair = (node[0] + row, node[1] + col)
        if pair not in loop_fine:
            neighbours.append(pair)
    return neighbours

## Code to see if we are outside of the original grid

In [10]:
min_row, min_col = 0, 0
max_row, max_col = len(data) - 1, len(data[1]) -1

def invalid_range(cur):
    row, col = cur
    if row < min_row or row > max_row:
        return True
    if col < min_col or col > max_col:
        return True
    return False

In [11]:
verified_non_isolated = set()

In [12]:
def walk_node(node):
    visited = set(node)
    queue = find_neighbours(node)
    isolated_node = True
    
    while len(queue) > 0:
        cur = queue.pop()
        
        if cur in verified_non_isolated or invalid_range(cur):
            isolated_node = False
            break
            
        visited.add(cur)

        for neighbour in find_neighbours(cur):
            if neighbour not in visited:
                queue.append(neighbour)
                
    # Populate the global set to speed up future searches
    if not isolated_node:
        for node in visited:
            verified_non_isolated.add(node)
    
    return isolated_node


In [13]:
res = []
for node in tqdm(all_locations):
    if node not in loop_fine:
        res.append(walk_node(node))
    
print("Answer #2:", sum(res))

  0%|          | 0/19600 [00:00<?, ?it/s]

Answer #2: 357
